In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parents[1]))
from model import left_right_task as lrt, network_model, util, plot_style

In [4]:
import os

print(os.getcwd())

c:\Users\caele\Documents\miller-lab-final\figures\figure7


In [5]:
# task parameters
numPairs = 5
dt = 1e-5
seq_len = 6
sequences = lrt.make_all_sequences(seq_len, ['L', 'R'])
equil_duration = 2

In [6]:
# load reliabilities
reliabilities = np.load('../figure6/data_1/reliabilities.npy', allow_pickle=True)
print(reliabilities.shape)

(1000,)


In [13]:
np.argsort(reliabilities)[-3:]

array([785, 797, 857], dtype=int64)

In [14]:
reliabilities[np.argsort(reliabilities)[-3:]]

array([1., 1., 1.])

In [9]:
Wji, pset, amp, dur, l_kernel, r_kernel = util.load_fiducial_network(True)

In [10]:
def run_trial(i, amp):
    rng = np.random.default_rng(i)
    
    # make kernels -- same mean
    l_kernel = rng.lognormal(0, 1, numPairs)
    l_kernel = l_kernel / np.sum(l_kernel) * numPairs
    l_kernel = np.round(l_kernel, 5)
    r_kernel = rng.lognormal(0, 1, numPairs)
    r_kernel = r_kernel / np.sum(r_kernel) * numPairs
    r_kernel = np.round(r_kernel, 5)
    
    # make stimuli
    stim_map = lrt.make_stim_map(numPairs, amp, dur, l_kernel, r_kernel, dt)
    
    # make FSM
    FSM = lrt.make_FSM(numPairs, pset, Wji, stim_map, 2, dt=dt)
    
    reliability = lrt.FSM_reliability(sequences, FSM)

    return reliability # for now
    
    # pcurve and decision dictionary computations -----------------------
    # trace all sequences on the graph
    nodes_dict = {node: [] for node in list(FSM.nodes)}
    for seq in sequences:
        # start at the beginning
        current_node = 1
        # trace the sequence
        for letter in seq:
            for edge in FSM.out_edges(current_node, keys=True):
                if FSM.edges[edge]['label'] == letter:
                    # move to the next node
                    current_node = edge[1]
                    break
        nodes_dict[current_node].append(seq)

    left_counts = np.zeros(seq_len + 1)
    left_choices = np.zeros(seq_len + 1)
    decision_dict = {}
    for node in nodes_dict:
        # is the node a L or R node? based on frequency
        l_cue_counts = [seq.count('L') for seq in nodes_dict[node]]
        num_above = sum([1 for count in l_cue_counts if count > seq_len/2])
        num_below = sum([1 for count in l_cue_counts if count < seq_len/2])
        if num_above == num_below:
            for count in l_cue_counts:
                left_counts[count] += 1     # record distribution of `counts` terminating in current node
                left_choices[count] += 0.5   # record how many times each `count` resulted in a left decision
            for seq in nodes_dict[node]:
                decision_dict[seq] = 'Tie'
        else:
            left_decision = num_below < num_above # True if L, False if R
            for count in l_cue_counts:
                left_counts[count] += 1     # record distribution of `counts` terminating in current node
                left_choices[count] += left_decision   # record how many times each `count` resulted in a left decision
            # update dictionary
            for seq in nodes_dict[node]:
                decision_dict[seq] = 'L' if left_decision else 'R'
    
    p_curve = left_choices/left_counts

    return reliability, p_curve, decision_dict

In [11]:
n_kernels = 3
amps = [amp/2, amp, amp*1.5]
new_reliabilities = np.zeros((len(amps), n_kernels))
for i, amp in tqdm(enumerate(amps)):
    for j, seed in enumerate(np.argsort(reliabilities)[-n_kernels:]):
        new_reliabilities[i, j] = run_trial(seed, amp)

3it [07:34, 151.63s/it]


In [12]:
new_reliabilities

array([[0.52272727, 0.52272727, 0.52272727],
       [0.79545455, 0.79545455, 1.        ],
       [0.52272727, 0.52272727, 0.68181818]])